[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/f_ivp/ivp.ipynb)

In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import sys

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Initial value problems

## Integrating the Rossler ODE system

The R\"ossler attractor is a chaotic ODE system with three coupled equations
$$
\begin{align*}
x' & = -y -z, \\
y' & = x+ay, \\
z' & = b+ z(x-c).
\end{align*}
$$
Consider solving this system using the trapezoid method,
$$
U^{n+1} - U^n = \frac{k\left[f(U^n,t_n) + f(U^{n+1},t_{n+1})\right]}{2}
$$
with $u(t) =
        (x(t),y(t),z(t))$. To evaluate $U^{n+1}$, we must find a root
of the equation
$$
F(U) = 2(U^n-U) + k \left[f(U^n,t_n) + f(U,t_{n+1})\right],
$$
where we will use Newton's method to perform the root-finding at each step.

### Problem setup

In [ ]:
# Constants in Rossler ODE system
a = 0.2
b = 0.2
c = 5.7

# Maximum number of Newton iterations
max_nsteps = 100

# Function to calculate derivatives x, y, and z in the Rossler attractor
def f(U, t):
    return np.array([-U[1]-U[2],
                     U[0]+a*U[1],
                     b+U[2]*(U[0]-c)])

# Function F to perform root-finding on, to compute the trapezoid update
# implicitly
def F(U, U_n, t, k):
    return 2*(U_n-U)+k*(f(U_n, t)+f(U, t+k))

# Jacobian of F, required for the Newton iteration
def J_F(U, t, k):
    return np.array([[-2, -k, -k],
                     [k, -2+a*k, 0],
                     [k*U[2], 0, -2+k*(U[0]-c)]])


# Initial condition, and step size
t = 0
U_n = np.array([-3., 0, 0])
k = 0.02

### Integrate ODE

In [ ]:
# Store the results for plotting
results = []

# Perform timesteps
print(t, U_n[0], U_n[1], U_n[2])
while t < 600:

    l = 0
    q = 0
    U = np.copy(U_n)

    # Perform Newton updates until two steps are below a tolerance
    while l < 2:

        # Calculate Newton iteration
        F_ = F(U, U_n, t, k)
        U -= np.linalg.solve(J_F(U, t, k), F_)

        # Check if the Newton iteration has converged
        if np.linalg.norm(F_) < 1e-12:
            l += 1
        else:
            l = 0

        # Check for too many iterations
        q += 1
        if q == max_nsteps:
            print("Too many Newton iterations")
            sys.exit()

    # Update solution using the result of the Newton iteration
    U_n = U
    t += k
    print(t, U_n[0], U_n[1], U_n[2], q)
    # Store results for plotting
    results.append([t, U_n[0], U_n[1], U_n[2]])

### Visualize

In [ ]:
# Extract time and x, y, z
results = np.array(results)
t = results[:, 0]
x = results[:, 1]
y = results[:, 2]
z = results[:, 3]

In [ ]:
# %matplotlib ipympl

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')

ax.plot(x, y, z, lw=0.5)
ax.scatter(x, y, z, c=t, cmap='viridis', s=0.1)

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

plt.show()


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

fig = plt.figure(figsize=(3, 3), dpi=50)
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
ax.set_xlim((x.min(), x.max()))
ax.set_ylim((y.min(), y.max()))
ax.set_zlim((z.min(), z.max()))

line, = ax.plot([], [], [], lw=0.5)
point, = ax.plot([], [], [], 'o', color='red')
time_text = ax.text2D(0.05, 0.95, '', transform=ax.transAxes)


def init():
    line.set_data([], [])
    line.set_3d_properties([])
    point.set_data([], [])
    point.set_3d_properties([])
    return line, point

def animate(frame):
    line.set_data(x[:frame], y[:frame])
    line.set_3d_properties(z[:frame])
    point.set_data([x[frame]], [y[frame]])
    point.set_3d_properties([z[frame]])
    return line, point

ani = FuncAnimation(fig, animate, frames=len(t), init_func=init, blit=True, interval=10)
plt.rcParams['animation.embed_limit'] = 1000

HTML(ani.to_jshtml())